In [ ]:
!pip -q install openai newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
import openai
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [ ]:
from enum import Enum
from typing import Union

from pydantic import BaseModel

import openai
from openai import OpenAI

## Article getter

In [ ]:
import newspaper

def download_article(url):
    # Create an Article object
    article = newspaper.Article(url)

    # Download and parse the article
    article.download()
    article.parse()

    # Get the article text
    article_text = article.text

    return article_text


In [ ]:

url = "https://techcrunch.com/2024/08/13/made-by-google-2024-a-few-ai-features-you-mightve-missed/"
article = download_article(url)

# Now, 'content' contains the text of the article
print("Article content:")
print(article[:500] + "...")

Article content:
Google’s annual Made by Google hardware event was held earlier Tuesday, and because the company announced so much stuff, a lot got lost in the shuffle. We rounded up some of the more intriguing AI-related announcements that didn’t get a ton of play, like the image-generating Pixel Studio and Pixel Screenshots, which helps save and organize info in screenshots.

Call Notes summarizes your conversations

For short-term-memory-challenged folks like this writer, Call Notes could be a useful feature....


### Setting up the Pydantic model

In [ ]:
from enum import Enum
from pydantic import BaseModel

class ProductType(str, Enum):
    """
    Enumeration of product types.

    This enum represents different categories of products that can be referenced.
    """
    device = "device"
    app = "app"
    ft = "fintech"
    saas = "saas"
    consumer_tech = "consumer_tech"

class OrgType(str, Enum):
    """
    Enumeration of product types.

    This enum represents different categories of products that can be referenced.
    """
    startup = "startup"
    big_tech = "big_tech"
    tech_media = "tech_media"
    social_media = "social_media"
    gov = "govt_org"
    non_prof = "non_profit_org"
    vc = "venture_captial"
    other = "other"

class Person(BaseModel):
    """
    Represents an individual associated with an organization.

    This model captures basic information about a person, including their name,
    the organization they're associated with, and their role within that organization.
    """
    name: str
    organization: str
    role: str

class Product(BaseModel):
    """
    Represents a product offered by an organization.

    This model captures basic information about a product, including its name,
    the organization it belongs to, and its type.
    """
    name: str
    organization: str
    product_type: ProductType

class Organization(BaseModel):
    """
    Represents an organization or company.

    This model captures basic information about an organization, including its name
    and location.
    """
    name: str
    org_type: OrgType
    location: str

class ArticleResponse(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates information about products, people, and organizations
    mentioned in an article, along with a summary of the article's content.
    """
    products: list[Product]
    people: list[Person]
    organizations: list[Organization]
    summary: str

In [ ]:
client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that scans for \
        people, products and organizations mentioned in articles."},
        {"role": "user", "content": article},
    ],
    response_format=ArticleResponse,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.people)
    for product in message.parsed.products:
        print(product)
    print(message.parsed.organizations)
    print(f"\n\nSummary: {message.parsed.summary}")
else:
    print(message.refusal)

[Person(name='Ivan Mehta', organization='Google', role='Colleague')]
name='Pixel 9' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel 9 Pro' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel 9 Pro Fold' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel Studio' organization='Google' product_type=<ProductType.app: 'app'>
name='Call Notes' organization='Google' product_type=<ProductType.app: 'app'>
name='Pixel Screenshots' organization='Google' product_type=<ProductType.app: 'app'>
[Organization(name='Google', org_type=<OrgType.big_tech: 'big_tech'>, location='Mountain View, California')]


Summary: Google's annual Made by Google hardware event unveiled several AI-related features exclusive to the Pixel 9 family of devices. These include Call Notes, which summarizes calls and saves details on the device; Pixel Studio, an image-generating app leveraging AI models both on-device and in the cloud; and Pixel S

In [ ]:
products = []
people = []
organizations = []

def parse_article(article):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that scans for \
            people, products and organizations mentioned in articles."},
            {"role": "user", "content": article},
        ],
        response_format=ArticleResponse,
    )

    message = completion.choices[0].message
    if message.parsed:
        print(message.parsed.people)
        for product in message.parsed.products:
            print(product)
        print(message.parsed.organizations)
        print(f"\n\nSummary: {message.parsed.summary}")
    else:
        print(message.refusal)

In [ ]:
url = "https://www.theverge.com/24218825/google-pixel-9-event-announcements-products"
article = download_article(url)
parse_article(article)

[Person(name='Chris Welch', organization='The Verge', role='Reporter'), Person(name='Victoria Song', organization='The Verge', role='Reporter')]
name='Pixel 9' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel 9 Pro' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel 9 Pro XL' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel 9 Pro Fold' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel Studio' organization='Google' product_type=<ProductType.app: 'app'>
name='Pixel Screenshots' organization='Google' product_type=<ProductType.app: 'app'>
name='Pixel Watch 3' organization='Google' product_type=<ProductType.device: 'device'>
name='Pixel Buds Pro 2' organization='Google' product_type=<ProductType.device: 'device'>
[Organization(name='Google', org_type=<OrgType.big_tech: 'big_tech'>, location='Mountain View, California'), Organization(name='The Verge', org_type=<OrgType.tech_media:

In [ ]:
url = "https://www.theverge.com/24219730/jeff-bezos-amazon-bbc-archive-shopping"
article = download_article(url)
parse_article(article)

[Person(name='Jeff Bezos', organization='Amazon', role='Founder and CEO'), Person(name='Nils Blythe', organization='BBC', role='Reporter')]
[Organization(name='Amazon', org_type=<OrgType.big_tech: 'big_tech'>, location='USA'), Organization(name='BBC', org_type=<OrgType.gov: 'govt_org'>, location='UK'), Organization(name="Sainsbury's", org_type=<OrgType.big_tech: 'big_tech'>, location='UK')]


Summary: The article recounts a 1997 BBC report that explored how the internet was expected to revolutionize shopping. It included interviews with local bakers, booksellers, and Jeff Bezos, discussing the potential of online commerce. At the time, a small UK bakery was successfully selling internationally online, while Sainsbury's had mixed results selling groceries using the internet. Fast forward to 2024, Amazon, founded by Jeff Bezos, reported $143.3 billion in revenue in the first quarter, showcasing the actualized potential of e-commerce years later.


In [ ]:
url = "https://techcrunch.com/2024/08/12/a16z-backed-fintech-tally-which-raised-172m-in-funding-is-shutting-down-after-running-out-of-cash/"
article = download_article(url)
parse_article(article)



[Person(name='Jason Brown', organization='Tally', role='Founder and CEO')]
name='Tally App' organization='Tally' product_type=<ProductType.app: 'app'>
[Organization(name='Tally', org_type=<OrgType.startup: 'startup'>, location='San Francisco'), Organization(name='Sway Ventures', org_type=<OrgType.vc: 'venture_captial'>, location=''), Organization(name='Andreessen Horowitz', org_type=<OrgType.vc: 'venture_captial'>, location=''), Organization(name='Kleiner Perkins', org_type=<OrgType.vc: 'venture_captial'>, location=''), Organization(name='Shasta Ventures', org_type=<OrgType.vc: 'venture_captial'>, location=''), Organization(name='Cowboy Ventures', org_type=<OrgType.vc: 'venture_captial'>, location=''), Organization(name='TechCrunch', org_type=<OrgType.tech_media: 'tech_media'>, location='')]


Summary: Tally, a fintech startup founded in 2015, has shut down after failing to secure necessary funding. The company, last valued at $855 million, initially aimed to help consumers manage cred